# Main notebook to run our application (neeeds 4GB of memory)

In [1]:
import pandas as pd
import networkx as nx
import math
import pickle
from itertools import islice
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType , DoubleType
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### 1. We load the preprocessd data

In [2]:
def load_hdfs_graph_data(): 
    spark = SparkSession.builder \
    .appName("Router") \
    .master("local") \
    .getOrCreate()

    stop_times_df =spark.read.parquet("../groupe-z/stop_times").toPandas()
    schema_walkable = StructType([
        StructField("from_stop_id", StringType(), nullable=True),
        StructField("to_stop_id", StringType(), nullable=True),
        StructField("min_transfer_time",FloatType(),nullable=True)
        # Add more fields as needed
    ])
    df_walkable_pairs = spark.read.schema(schema_walkable).parquet("../groupe-z/walkable_transfers").toPandas()
    schema_dict = StructType([
    StructField("stop_id", StringType(), nullable=True),
    StructField("stop_name", StringType(), nullable=True),
    StructField("stop_lat", DoubleType(), nullable=True),
    StructField("stop_lon", DoubleType(), nullable=True),
    StructField("location_type", StringType(), nullable=True),
    StructField("parent_station", StringType(), nullable=True),
    StructField("distance_to_zurich_hb", DoubleType(), nullable=True),
])

    delays_df = spark.read.parquet("../groupe-z/delays").toPandas()

    return stop_times_df , df_walkable_pairs  , delays_df





stop_times_df , df_walkable_pairs  ,delays_df = load_hdfs_graph_data()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# load local data
id_to_name_dict  = pickle.load(open("../data/id_to_name_dict.pickle","rb"))

name_to_id_dict  = pickle.load(open("../data/name_to_id_dict.pickle","rb"))

stops_df = pickle.load(open("../data/stops_df.pickle","rb"))


dict_walkable_pairs = {(row["from_stop_id"],row["to_stop_id"]):row["min_transfer_time"] for _,row in df_walkable_pairs.iterrows()}

code_to_type = {'walk': '', 
                'IC': 'Zug',
                'IR': 'Zug',
                'ICE': 'Zug',
                'TGV': 'Zug',
                'RE': 'Zug',
                'NJ': 'Zug',
                'RJX': 'Zug',
                'B': 'Bus',
                'T': 'Tram',
                'FUN': 'Metro',
                'BAT': '',
                'FAE': '',
                'S': 'Zug',
                'EXT': '',
                'FAE': '',
                'EC': 'Zug',
                'PB':'Bus'
}


# transform the data into an edge dataframe
df_times = pd.DataFrame(stop_times_df["attr"].to_list(), columns=['src_arrival_time','src_departure_time', 'dst_arrival_time', 'dst_departure_time'])
df_edges = pd.concat([stop_times_df, df_times], axis=1)
df_edges = df_edges.drop(columns=['attr','dst_departure_time'])
df_w = df_walkable_pairs.rename(columns={"from_stop_id": "src", "to_stop_id": "dst","min_transfer_time":"dst_arrival_time"})
df_w["src_arrival_time"] = -1
df_w["src_departure_time"] = 0
df_w['trip_id'] = [i+1 for i in range(len(df_w))]
df_edges = pd.concat([df_edges, df_w], axis=0)
df_edges = df_edges.reset_index(drop=True)
df_edges["travel_time"] = df_edges["dst_arrival_time"] - df_edges["src_departure_time"]


df_edges_grouped = df_edges.groupby(['src','dst',"src_departure_time","src_arrival_time","dst_arrival_time"]).agg(list).reset_index()
df_edges_grouped["weight"]= df_edges_grouped["travel_time"].apply(lambda x: min(x))
df_edges_grouped["num_trips"]= df_edges_grouped["travel_time"].apply(lambda x: len(x))
df_edges_grouped["type"]= df_edges_grouped["transport_type"].apply(lambda x: x[0])
df_edges_grouped["type"]= df_edges_grouped["type"].fillna("walk")
df_edges_grouped["trip_id"]= df_edges_grouped["trip_id"].apply(lambda x: x[0])
df_edges_filtered = df_edges_grouped[df_edges_grouped["weight"]>0][["src","dst","weight","src_arrival_time","src_departure_time","dst_arrival_time","type","trip_id"]]
df_edges_filtered['type']  = df_edges_filtered['type'].apply(lambda x : code_to_type[x])
df_edges_filtered.to_pickle("../data/df_edges_filtered.pickle")

In [35]:
def get_sec(time_str):
    """Get seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)
def get_time(seconds):
    """Get time from seconds."""
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%02d:%02d:%02d" % (h, m, s)
def get_time_beautified(time_str):
    h,m,s = time_str.split(":")
    h,m,s=int(h),int(m),int(s)
    hours_str = "" if h ==0 else (f"{h}hr")
    minutes_str = "" if m == 0 else (f"{m}min")
    return hours_str + minutes_str
    
    

def walkable(src,dst):
    return (src,dst) in dict_walkable_pairs.keys(), dict_walkable_pairs.get((src,dst),-1)


### 2. find paths

In [36]:
def find_potential_nodes(source, target, k):
    G = nx.from_pandas_edgelist(df_edges_filtered, 'src', 'dst', ['weight'], create_using=nx.DiGraph())
    paths = list(islice(nx.shortest_simple_paths(G, source, target, weight='weight'), k))
    flat_list = list(set([item for sublist in paths for item in sublist]))
    return flat_list


In [37]:
MAX_WAIT_TIME = 1800
CONNECTION_WINDOW = 7200

def find_actual_connections_arrival_time(start,end,desired_arrival_time,tolerance=1800,k=10,verbose=False):
    print("finding potential stops") if verbose else None
    potential_nodes = find_potential_nodes(start,end,100)
    pairs = [ (src,dst) for src in potential_nodes for dst in potential_nodes if src!=dst]
    df_pairs = pd.DataFrame(pairs,columns=["src","dst"])
    df_path = df_edges_filtered.merge(df_pairs,on=["src","dst"],how="inner")
    df_path = df_path[(df_path["type"]!="walk") & (df_path["src_arrival_time"].apply(lambda x : abs(x - desired_arrival_time)<CONNECTION_WINDOW))]
    df_path["connection_id"] = range(len(df_path))

    
    ###################### 1. Create the graph ######################
    H = nx.DiGraph()

    print("adding nodes") if verbose else None
    #First, we add the nodes, each node corresponds to a connection between two stops
    for stop in potential_nodes:
        connections = df_path[(df_path["src"]==stop)]
        connections = connections[~connections.duplicated(subset=connections.columns.difference(['trip_id']))]
        for _,row in connections.iterrows():
            H.add_node(row["connection_id"],src_arrival_time=row["src_arrival_time"],src_departure_time=row["src_departure_time"],dst_arrival_time=row["dst_arrival_time"],weight=row["weight"],src=row["src"],dst=row["dst"],type=row["type"],trip_id=row['trip_id'])
    #Then, we add the edges, an edge exists if it is possible to take a connection from one stop to another
    print("adding edges") if verbose else None
    nodes = list(H.nodes(data=True))
    for src_index, src_attr in nodes:
        for dst_index,dst_attr in nodes:
            if src_index == dst_index:
                continue
            if src_attr["dst"] != dst_attr["src"]: 
                is_walkable, walking_time = walkable(src_attr["dst"], dst_attr["src"])
                is_connection_possible = (src_attr["dst_arrival_time"] + walking_time < dst_attr["src_departure_time"])
                is_wait_not_too_long = (src_attr["dst_arrival_time"] - src_attr["src_departure_time"] < MAX_WAIT_TIME)
                if (is_walkable and is_wait_not_too_long and is_connection_possible):
                    H.add_edge(src_index,dst_index,weight=dst_attr["src_departure_time"]-src_attr["src_departure_time"])
            else :
                is_connection_possible = (src_attr["dst_arrival_time"] < dst_attr["src_departure_time"]) or (abs(src_attr["dst_arrival_time"] - dst_attr["src_departure_time"])<1)
                is_wait_not_too_long =  (dst_attr["src_departure_time"] - src_attr["dst_arrival_time"] < MAX_WAIT_TIME)
                if (is_connection_possible and is_wait_not_too_long):
                    weight = dst_attr["src_departure_time"] - src_attr["src_departure_time"]
                    H.add_edge(src_index,dst_index,weight=weight)
                
    #We add the end and start nodes, these will represent the start and end of the route ()
    #This allows the route to start from any time 
    connections_from_start = [(node,attr) for node, attr in H.nodes(data=True) if (attr["src"]==start or walkable(start,attr["src"])[0])]
    if len(connections_from_start) == 0:

        print("no connections found for departure") if verbose else None
        return []
    for node,attr in connections_from_start:
        H.add_edge("start",node,weight=0)

    
    #This allows the route to end only at the desired time, with a tolerance defined as a hyperparameter
    connections_to_dest = [(node,attr) for node, attr in H.nodes(data=True) if node!="start" and (attr["dst"]==end or walkable(attr["dst"],end)[0]) and (attr["dst_arrival_time"] < desired_arrival_time) and (attr["dst_arrival_time"] > (desired_arrival_time-tolerance))]
    if len(connections_to_dest) == 0:
        print("no connections found for destination") if verbose else None
        return []
    for node,attr in connections_to_dest:
        print("possible arrival time  from connection", attr["dst"],"at",get_time(attr["dst_arrival_time"])) if verbose else None
        if attr["dst"] != end:
            is_walkable, walking_time = walkable(attr["dst"],end)
            H.add_edge(node,"end",weight=walking_time)
        else:
            H.add_edge(node,"end",weight=attr["weight"])


    ###################### 2. Find the k shortest paths ######################
    final_paths  = []
    for _ in range(k):
        if not nx.has_path(H,"start","end"):
            break
        path = nx.shortest_path(H, "start", "end",weight="weight")
        res = path[1:-1]
        departure_time = H.nodes[res[0]]["src_departure_time"]
        arrival_time = H.nodes[res[-1]]["dst_arrival_time"]
        print("departure_time",get_time(departure_time)) if verbose else None
        print("arrival_time",get_time(arrival_time)) if verbose else None
        print("total time spent",get_time(arrival_time-departure_time)) if verbose else None
        waypoints=[]
        transport_order = 0
        last_trip_id = H.nodes[res[0]]["trip_id"]
        for waypoint in res:

            src_departure_time = H.nodes[waypoint]["src_departure_time"]
            dst_arrival_time = H.nodes[waypoint]["dst_arrival_time"]
            src = H.nodes[waypoint]["src"]
            dst = H.nodes[waypoint]["dst"]
            type_ = H.nodes[waypoint]["type"]
            trip_id = H.nodes[waypoint]["trip_id"]
            src_name = id_to_name_dict[src] if (src in id_to_name_dict.keys()) else src
            dst_name = id_to_name_dict[dst] if (dst in id_to_name_dict.keys()) else dst
            if last_trip_id != trip_id:
                transport_order +=1
                last_trip_id=trip_id
            w = {"src_departure_time":get_time(src_departure_time),"dst_arrival_time":get_time(dst_arrival_time),"src":src_name,"dst":dst_name,"type":type_,"trip_id":trip_id,"src_id":src,"dst_id":dst,"transport_order":transport_order}
            print(w) if verbose else None
            waypoints.append(w)
        final_paths.append({"departure_time":get_time(departure_time),"arrival_time":get_time(arrival_time),"travel_time":arrival_time-departure_time,"waypoints":waypoints})
        H.remove_edge(path[-2],path[-1])
    return final_paths







In [7]:
def route (start,end, desired_arrival_time, verbose = False):
    actual_paths = find_actual_connections_arrival_time(start,end,get_sec(desired_arrival_time),verbose=verbose)
    print("sorting found paths") if verbose else None
    actual_paths = sorted(actual_paths,key=lambda x: get_sec(x["arrival_time"]))
    return actual_paths

### 3 . Adding delays 

In [8]:
# extracted from notebook 02_calculating_delays
AVG_DEP_DELAY = 108.6383003197186
AVG_ARR_DELAY =  78.77420048977521

STD_DEP_DELAY = 108.53734109482895
STD_ARR_DELAY = 89.98687439461523

In [91]:
import scipy.stats as stats
import scipy.integrate as integrate
import numpy as np

def compute_confidence(path, verbose=True): 
    
    # create dataframe for the path
    path_df = pd.DataFrame(path['waypoints'])
    
    display(path_df) if verbose else None
    
    # different trips
    df_trips = path_df.groupby('trip_id').agg(list)
    

    df_trips['first'] = df_trips['src'].apply(lambda x: x[0])
    df_trips['last'] = df_trips['dst'].apply(lambda x: x[-1])
    df_trips['first_id'] = df_trips['src_id'].apply(lambda x: x[0])
    df_trips['last_id'] = df_trips['dst_id'].apply(lambda x: x[-1])

    df_trips['last_arrival_time'] = df_trips['dst_arrival_time'].apply(lambda x: x[-1])
    df_trips['first_departure_time'] = df_trips['src_departure_time'].apply(lambda x: x[0])
    df_trips['type'] = df_trips['type'].apply(lambda x : x[0])
    df_trips = df_trips.sort_values(by='first_departure_time')
    df_trips.reset_index(inplace=True)
    
   
        # probability of not missing any connection
    p_all = 1
    connections_info = []
    for i in range(len(df_trips)-1): 
        last_src_name = df_trips.iloc[i]['last']
        first_dst_name = df_trips.iloc[i+1]['first']
        last_src_id = df_trips.iloc[i]['last_id']
        first_dst_id = df_trips.iloc[i+1]['first_id']
        last_arrival_time = df_trips.iloc[i]['last_arrival_time']
        first_departure_time = df_trips.iloc[i+1]['first_departure_time']
        last_type = df_trips.iloc[i]['type']
        first_type = df_trips.iloc[i+1]['type']
        # convert time to datetime
        last_arrival_time = pd.to_datetime(last_arrival_time, format='%H:%M:%S')
        first_departure_time = pd.to_datetime(first_departure_time, format='%H:%M:%S')



        print(f"""********connection{i}*******""") if (verbose) else None
        print(f"""from_name = {last_src_name}, to_name = {first_dst_name}, from={last_src_id}, to={first_dst_id}, start_time = {last_arrival_time}, deadline = {first_departure_time}""") if (verbose) else None

        if last_src_name == first_dst_name: # NOT A WALK EDGE (simply train change)
            delay_max = (first_departure_time - last_arrival_time).seconds
            transfer = 0 
        else : # a walk edge
            has_transfer_time = len(df_walkable_pairs[(df_walkable_pairs['from_stop_id'] == last_src_id) & (df_walkable_pairs['to_stop_id'] == first_dst_id)]['min_transfer_time'].values) > 0
            transfer = 0 if not has_transfer_time else df_walkable_pairs[(df_walkable_pairs['from_stop_id'] == last_src_id) & (df_walkable_pairs['to_stop_id'] == first_dst_id)]['min_transfer_time'].values[0]
            print('transfer = ', transfer) if (verbose) else None
            delay_max = (first_departure_time - last_arrival_time).seconds - transfer

        print('delay_max = ', delay_max) if (verbose) else None

               # get the delay of arrival for last stop of the first trip
        has_arrival_delay_avg = len(delays_df[(delays_df['stop_name'] == last_src_name ) & (delays_df['transport_type'] == last_type) ]['avg_arr_delay'].values) > 0
        last_arrival_delay_avg = pd.NA if not has_arrival_delay_avg else delays_df[(delays_df['stop_name'] == last_src_name ) & (delays_df['transport_type'] == last_type) ]['avg_arr_delay'].values[0]
        last_arrival_delay_avg = last_arrival_delay_avg if not pd.isna(last_arrival_delay_avg) else AVG_ARR_DELAY

        last_arrival_delay_stddev = pd.NA if not has_arrival_delay_avg else delays_df[(delays_df['stop_name'] == last_src_name ) & (delays_df['transport_type'] == last_type)]['stddev_arr_delay'].values[0]
        last_arrival_delay_stddev = last_arrival_delay_stddev if not pd.isna(last_arrival_delay_stddev) else STD_ARR_DELAY

        # get the delay of departure for the first stop of the second trip
        first_departure_delay_avg = pd.NA if not has_arrival_delay_avg else  delays_df[(delays_df['stop_name'] == first_dst_name ) & (delays_df['transport_type'] == first_type)]['avg_dep_delay'].values[0]
        first_departure_delay_avg = first_departure_delay_avg if not pd.isna(first_departure_delay_avg) else AVG_DEP_DELAY

        first_departure_delay_stddev =  pd.NA if not has_arrival_delay_avg else delays_df[(delays_df['stop_name'] == first_dst_name ) & (delays_df['transport_type'] == first_type)]['stddev_dep_delay'].values[0]
        first_departure_delay_stddev = first_departure_delay_stddev if not pd.isna(first_departure_delay_stddev) else STD_DEP_DELAY



        ### METHOD 1 : using the probability of delay > delay_max
        # probability of delay > delay_max
        p = 1 - stats.norm.cdf(delay_max, last_arrival_delay_avg, last_arrival_delay_stddev)
        print('average_delay = ', last_arrival_delay_avg) if (verbose) else None
        print('stddev_delay = ', last_arrival_delay_stddev) if (verbose) else None
        print('p = ', p) if (verbose) else None


        ### METHOD 2 : using the probability of delay > delay_max-y knowing y (y is the delay of departure of the second trip)
        # Define the integrand function for p_sachant
        def integrand(delay_dep):
            p_dep = stats.norm.pdf(delay_dep, first_departure_delay_avg, first_departure_delay_stddev)
            return (1 - stats.norm.cdf(delay_max + delay_dep, last_arrival_delay_avg, last_arrival_delay_stddev)) * p_dep

        p_sachant, _ = integrate.quad(integrand, -np.inf, np.inf)  
        print('p sachant = ', p_sachant) if (verbose) else None
        
        connections_info.append({'proba_miss' : p_sachant , 'src' : last_src_name , 'dst': first_dst_name , 'transfer_time' : transfer , 'rank_con' : i })


        # print general info
        print('probability of missing this connection = ', p_sachant) if (verbose) else None

        p_all = (1 - p_sachant) * p_all

    print('probability of not missing any connection = ', p_all) if (verbose) else None
    
    return p_all , connections_info




    

In [77]:
def add_delays(paths,verbose=False):
    for i,path  in enumerate(paths) : 

        proba, info = compute_confidence(path,verbose=verbose)

        path['confidence'] = proba
        path['confidence_info'] = info


### 4. Visualization

In [78]:
def create_map(journey,i):
    colors = ["yellow","red","white","green","purple","orange",
          "blue",  
          "brown",
            "cyan",
            "navy",
            "linen",
            "tomato",
            "olive",
            "lime",
            "greenyellow"]
    j_dep_time = journey['departure_time']
    j_arr_time = journey['arrival_time']
    j_duration = get_time(journey['travel_time'])
    waypoints = journey['waypoints']


    
    df = pd.DataFrame.from_records(waypoints)
    #add long and lat
    df["src_lon"] = df.merge(stops_df,left_on="src_id",right_on="stop_id")["stop_lon"]
    df["src_lat"] = df.merge(stops_df,left_on="src_id",right_on="stop_id")["stop_lat"]
    df["dst_lon"] = df.merge(stops_df,left_on="dst_id",right_on="stop_id")["stop_lon"]
    df["dst_lat"] = df.merge(stops_df,left_on="dst_id",right_on="stop_id")["stop_lat"]
    
    last_row = df.iloc[-1]
    first_row = df.iloc[0]
    dst_lon,dst_lat,dst_name, = last_row["dst_lon"],last_row["dst_lat"],last_row["dst"]
    src_lon,src_lat = first_row["src_lon"],first_row["src_lat"]
    orders = sorted(df["transport_order"].unique())
    
    
        # l is the layout of the plot, we use to set the title, the margins and the style of the map
    l = go.Layout(
        title= f'Journey {i+1}: Departure: {j_dep_time} Arrival: {j_arr_time}', 
        margin ={'l':0,'t':100,'b':0,'r':0},
        mapbox = {
            'style': "open-street-map",
            'center': {'lon':(dst_lon+src_lon)/2 , 'lat': (dst_lat+src_lat)/2},
            'zoom': 9,},
        title_x = 0.41
    )

    fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines",
            marker = {'size': 10},
            ), l)
    
    
    for _,row in df.iterrows():
        fig.add_trace(go.Scattermapbox(mode='lines',
                                   lon=[row['src_lon'], row['dst_lon']],
                                   lat=[row['src_lat'], row['dst_lat']],
                                   line_color=colors[row["transport_order"]],
                                   name=row['dst'] 
        ))  

    fig.add_trace(go.Scattermapbox(mode = "markers",
                                    lon = df["src_lon"].append(pd.Series([dst_lon])),
                                    lat = df["src_lat"].append(pd.Series([dst_lat])),
                                    text= df["src"].append(pd.Series([dst_name])),
                                    customdata=df["src_departure_time"].append(pd.Series([""])),
                                    showlegend=False,
                                    marker = {'size': 12,'color': "black"},
                                    hovertemplate="<br>".join([
    "Station name: %{text}","Depature time: %{customdata}"])))
    return fig

In [86]:
def build_paths_accordion(journeys):
    titles=[]
    children =[]
    
    for i,journey in enumerate(journeys):
        j_dep_time = journey['departure_time']
        j_arr_time = journey['arrival_time']
        j_duration = get_time(journey['travel_time'])
        j_confidence = journey.get("confidence")
        j_confidence_info = journey.get("confidence_info")
        titles.append(f"{j_dep_time[:-3]}  =====> {j_arr_time[:-3]} \t\t\t\t\t\t{get_time_beautified(j_duration)} \t\t\t\t\t\t Q: {(j_confidence*100):.2f}%")
        
        
        
        transfers_labels = []
        transfers_labels.append(widgets.Label(value=f"There {'is' if len(j_confidence_info) == 1 else 'are'} {len(j_confidence_info)} transfer{'' if len(j_confidence_info) == 1 else 's' } for this trip"))
        for transfer in j_confidence_info:
            transfers_labels.append(widgets.Label(value=f"\t- You have a probabiltiy of {(100*transfer['proba_miss']):.2f}% of missing your connection between {transfer['src']} and {transfer['dst']}"))
        
        children.append(widgets.VBox(transfers_labels))
        
    accordion = widgets.Accordion(children=children, titles=titles)
    return accordion

In [88]:
# Below we define all the widgets
import datetime
import warnings
import sys

warnings.simplefilter(action='ignore', category=FutureWarning)

time_picker = widgets.TimePicker(
    description='Pick a Time',
    value= datetime.datetime.strptime("10:00", '%H:%M'),
    disabled=False
)

departure_station = widgets.Combobox(
    value= "Hausen am Albis, Riedmatt",
    placeholder='Type station name',
    options=list(name_to_id_dict.keys()),
    description='Departure Station:',
    ensure_option=True,
    disabled=False
)


arrival_station = widgets.Combobox(
    value = "Zürich Flughafen",
    placeholder='Type station name',
    options=list(name_to_id_dict.keys()),
    description='Arrival Station:',
    ensure_option=True,
    disabled=False
)

layout = widgets.Layout(width='auto', height='50px')

button = widgets.Button(
    description='Find route',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button',
    icon='search', # (FontAwesome names without the `fa-` prefix)
    layout = layout
)


out = widgets.Output()

# The following functions defines what happens when we press the button

                             
def button_action(button):
    
    out.clear_output()   # each time we press the button we clear the previous output
    
    with out:

        dep_station_id = name_to_id_dict.get(departure_station.value)
        arr_station_id = name_to_id_dict.get(arrival_station.value)
                
        if dep_station_id is None or arr_station_id is None:
            print("You need to select a start and destination", file=sys.stderr)
            return
        if dep_station_id == arr_station_id:
            print("Start and end station shoud not be the same", file=sys.stderr)
            return
        
        time_picker_string = time_picker.value
        if time_picker_string is None:
            print("You must choose an arrival time", file=sys.stderr)
            return
        desired_arrival_time = (time_picker_string.strftime("%H:%M:%S"))
        
        
        progress_bar = widgets.FloatProgress(
            value=0,
            min=0,
            max=10.0,
            description='Starting...',
            bar_style='info',
            orientation='horizontal'
        )
        display(progress_bar)
        
        progress_bar.value = 5
        progress_bar.description = "Calculating paths"
        paths = route(dep_station_id,arr_station_id,desired_arrival_time)
        
        # keep only distinct routes
        
        if len(paths) == 0:
            print("No paths found", file=sys.stderr)
            return
        progress_bar.value = 7
        progress_bar.description = "Adding delays"
        add_delays(paths,verbose=False)
        
        progress_bar.value = 8
        progress_bar.description = "Creating maps"
        
        maps = [create_map(path,i) for (i,path) in enumerate(paths)]
        
        output_maps = widgets.Output()
        accordion = build_paths_accordion(paths)

        def accordion_eventhandler(change):
            with output_maps:
                clear_output()
                i = change.new
                if i is not None:
                    display(maps[i])
        accordion.observe(accordion_eventhandler, names='selected_index')

        # Display the dropdown widget and initial figure
        display(accordion)
        display(output_maps)
        
        progress_bar.value = 10
        progress_bar.description = "Done!"
        
        
button.on_click(button_action)

In [89]:
display(time_picker , departure_station, arrival_station, button, out)

TimePicker(value=datetime.datetime(1900, 1, 1, 10, 0), description='Pick a Time', step=60.0)

Combobox(value='Hausen am Albis, Riedmatt', description='Departure Station:', ensure_option=True, options=('Oe…

Combobox(value='Zürich Flughafen', description='Arrival Station:', ensure_option=True, options=('Oetwil a.d.L.…

Button(description='Find route', icon='search', layout=Layout(height='50px', width='auto'), style=ButtonStyle(…

Output()